In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import time
from tqdm import tqdm
import h2o
from h2o.automl import H2OAutoML

%matplotlib inline
h2o.init(nthreads=8)
h2o.no_progress()

np.random.seed(42)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
  Starting server from D:\Anaconda3\envs\electric\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\delab_2\AppData\Local\Temp\tmpovglh9o_
  JVM stdout: C:\Users\delab_2\AppData\Local\Temp\tmpovglh9o_\h2o_delab_2_started_from_python.out
  JVM stderr: C:\Users\delab_2\AppData\Local\Temp\tmpovglh9o_\h2o_delab_2_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_delab_2_88klhr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26.67 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
version='v2'

# Data Load

In [3]:
train = pd.read_csv("./data/train.csv", encoding='cp949')

In [4]:
train.describe()

,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122400 entries, 0 to 122399
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   num         122400 non-null  int64  
 1   date_time   122400 non-null  object 
 2   전력사용량(kWh)  122400 non-null  float64
 3   기온(°C)      122400 non-null  float64
 4   풍속(m/s)     122400 non-null  float64
 5   습도(%)       122400 non-null  float64
 6   강수량(mm)     122400 non-null  float64
 7   일조(hr)      122400 non-null  float64
 8   비전기냉방설비운영   122400 non-null  float64
 9   태양광보유       122400 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 9.3+ MB


In [6]:
test = pd.read_csv("./data/test.csv", encoding='cp949')

In [7]:
test.describe()

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
count,10080.000000,3360.000000,3360.000000,3360.000000,1680.000000,3360.000000,2296.0,1624.0
mean,30.500000,27.805804,2.440327,81.963095,2.191845,0.612917,1.0,1.0
std,17.318961,2.378122,1.835905,11.900843,6.786772,0.929587,0.0,0.0
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,1.0,1.0
25%,15.750000,26.000000,1.100000,74.000000,0.000000,0.000000,1.0,1.0
50%,30.500000,27.500000,2.100000,83.000000,0.000000,0.000000,1.0,1.0
75%,45.250000,29.200000,3.200000,92.000000,0.900000,1.000000,1.0,1.0
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.0,1.0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   num           10080 non-null  int64  
 1   date_time     10080 non-null  object 
 2   기온(°C)        3360 non-null   float64
 3   풍속(m/s)       3360 non-null   float64
 4   습도(%)         3360 non-null   float64
 5   강수량(mm, 6시간)  1680 non-null   float64
 6   일조(hr, 3시간)   3360 non-null   float64
 7   비전기냉방설비운영     2296 non-null   float64
 8   태양광보유         1624 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 708.9+ KB


# Feature Engineering

In [9]:
#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])
train['time'] = train['date_time'].apply(lambda x: time(x))
test['time'] = test['date_time'].apply(lambda x: time(x))

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train['weekday'] = train['date_time'].apply(lambda x :weekday(x))
test['weekday'] = test['date_time'].apply(lambda x :weekday(x))

def month(x):
    return int(x[5:7])
train['month'] = train['date_time'].apply(lambda x :month(x))
test['month'] = test['date_time'].apply(lambda x :month(x))

In [10]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice = {}
hot = {}
count = 0
for i in tqdm(range(0, len(train), len(train)//60)):
    count += 1
    ice[count] = train.loc[i,'비전기냉방설비운영']
    hot[count] = train.loc[i,'태양광보유']

100%|████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 7491.17it/s]


In [11]:
for i in tqdm(range(len(test))):
    test.loc[i, '비전기냉방설비운영'] = ice[test['num'][i]]
    test.loc[i, '태양광보유'] = hot[test['num'][i]]

100%|███████████████████████████████████████████████████████████████████████████| 10080/10080 [00:16<00:00, 605.78it/s]


In [18]:
test = test.interpolate(method='values')

In [ ]:
test

In [12]:
def DI(temp, humid):
    '''
    disscomfort index : 불쾌지수
    '''
    return 0.81 * temp +0.01 * humid * (0.99*temp - 14.3)+ 46.3

In [13]:
# DI 80 이상이면 50% 정도 불쾌감을 느낌
train['DI'] = DI(train['기온(°C)'], train['습도(%)'])
train['feel_DI'] = train['DI'].map(lambda x : 1 if x > 80 else 0)

test['DI'] = DI(test['기온(°C)'], train['습도(%)'])
test['feel_DI'] = test['DI'].map(lambda x : 1 if x > 80 else 0)

In [14]:
train['태양광'] =  train['일조(hr)'] * train['태양광보유'] * train['강수량(mm)'].map(lambda x : np.exp(-x)) * train['풍속(m/s)'] / train['습도(%)'] * 100
test['태양광'] =  test['일조(hr, 3시간)'] * test['태양광보유'] * test['강수량(mm, 6시간)'].map(lambda x : np.exp(-x)) * test['풍속(m/s)'] / test['습도(%)'] * 100

In [15]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,month,DI,feel_DI,태양광
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,6,63.43008,0,0.000000
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,6,63.56993,0,0.000000
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,6,63.22775,0,0.000000
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,0,6,62.54339,0,0.000000
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,0,6,62.39760,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,19,0,8,77.80896,0,2.367647
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,20,0,8,77.44917,0,0.000000
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,21,0,8,77.44917,0,0.000000
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,22,0,8,77.52246,0,0.000000


In [16]:
train.describe()

,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유,time,weekday,month,DI,feel_DI,태양광
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333,11.500000,2.964706,6.929412,73.461061,0.079779,0.418172
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724,6.922215,2.014351,0.793925,4.778330,0.270952,1.377256
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,52.344210,0.000000,0.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000,5.750000,1.000000,6.000000,70.035960,0.000000,0.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000,11.500000,3.000000,7.000000,73.615850,0.000000,0.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000,17.250000,5.000000,8.000000,77.074510,0.000000,0.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000,23.000000,6.000000,8.000000,86.661790,1.000000,20.000000


In [17]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday,month,DI,feel_DI,태양광
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0.0,0.0,0,1,8,80.98224,1,0.0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1,1,8,NaN,0,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2,1,8,NaN,0,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0.0,0.0,3,1,8,79.99457,0,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4,1,8,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1.0,1.0,19,0,8,NaN,0,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1.0,1.0,20,0,8,NaN,0,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0,21,0,8,80.22185,1,NaN
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1.0,1.0,22,0,8,NaN,0,NaN


In [19]:
test.describe()

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유,time,weekday,month,DI,feel_DI,태양광
count,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.000000,10080.00000,10080.000000,10080.0,10080.000000,10080.000000,10080.000000
mean,30.500000,27.805813,2.440585,81.962500,2.191845,0.612917,0.683333,0.483333,11.50000,3.000000,8.0,79.101515,0.118452,1.153618
std,17.318961,2.275416,1.785098,11.467641,5.704649,0.867991,0.465199,0.499747,6.92253,2.000099,0.0,3.705743,0.323159,3.084974
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,8.0,69.795700,0.000000,0.000000
25%,15.750000,26.133333,1.200000,74.666667,0.000000,0.000000,0.000000,0.000000,5.75000,1.000000,8.0,76.558027,0.000000,0.000000
50%,30.500000,27.466667,2.100000,83.000000,0.000000,0.100000,1.000000,0.000000,11.50000,3.000000,8.0,78.620090,0.000000,0.000000
75%,45.250000,29.100000,3.175000,91.000000,1.500000,1.033333,1.000000,1.000000,17.25000,5.000000,8.0,81.195246,0.000000,0.213211
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.000000,1.000000,23.00000,6.000000,8.0,93.681380,1.000000,30.196078


mini_target = train.loc[:,['전력사용량(kWh)']].min()
size_target = train.loc[:,['전력사용량(kWh)']].max()-train.loc[:,['전력사용량(kWh)']].min()
train.loc[:,['전력사용량(kWh)']] = (train.loc[:,['전력사용량(kWh)']]-mini_target)/size_target

mini_input1 = train.loc[:,['기온(°C)']].min()
size_input1 = train.loc[:,['기온(°C)']].max()-train.loc[:,['기온(°C)']].min()
train.loc[:,['기온(°C)']] = (train.loc[:,['기온(°C)']]-mini_input1)/size_input1

mini_input2 = train.loc[:,['풍속(m/s)']].min()
size_input2 = train.loc[:,['풍속(m/s)']].max()-train.loc[:,['풍속(m/s)']].min()
train.loc[:,['풍속(m/s)']] = (train.loc[:,['풍속(m/s)']]-mini_input2)/size_input2

mini_input3 = train.loc[:,['습도(%)']].min()
size_input3 = train.loc[:,['습도(%)']].max()-train.loc[:,['습도(%)']].min()
train.loc[:,['습도(%)']] = (train.loc[:,['습도(%)']]-mini_input3)/size_input3

mini_input4 = train.loc[:,['강수량(mm)']].min()
size_input4 = train.loc[:,['강수량(mm)']].max()-train.loc[:,['강수량(mm)']].min()
train.loc[:,['강수량(mm)']] = (train.loc[:,['강수량(mm)']]-mini_input4)/size_input4

mini_input5 = train.loc[:,['강수량(mm)']].min()
size_input5 = train.loc[:,['강수량(mm)']].max()-train.loc[:,['강수량(mm)']].min()
train.loc[:,['강수량(mm)']] = (train.loc[:,['강수량(mm)']]-mini_input5)/size_input5

mini_input6 = train.loc[:,['time']].min()
size_input6 = train.loc[:,['time']].max()-train.loc[:,['time']].min()
train.loc[:,['time']] = (train.loc[:,['time']]-mini_input6)/size_input6

mini_input7 = train.loc[:,['weekday']].min()
size_input7 = train.loc[:,['weekday']].max()-train.loc[:,['weekday']].min()
train.loc[:,['weekday']] = (train.loc[:,['weekday']]-mini_input7)/size_input7

# Feature Selection

In [20]:
train_x = train.drop(['date_time'], axis=1)

In [21]:
train_x = train_x.rename({'전력사용량(kWh)':'feature_1', '기온(°C)': 'feature_2', '풍속(m/s)':'feature_3', '습도(%)':'feature_4', '강수량(mm)':'feature_5', '일조(hr)':'feature_6', '비전기냉방설비운영':'feature_7', '태양광보유':'feature_8', 'time':'feature_9', 'weekday':'feature_10', 'month':'feature_11', 'DI':'feature_12', 'feel_DI':'feature_13', '태양광':'feature_14'}, axis='columns')

In [22]:
train_x

,num,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,6,63.43008,0,0.000000
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,6,63.56993,0,0.000000
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,6,63.22775,0,0.000000
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,0,6,62.54339,0,0.000000
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,0,6,62.39760,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,19,0,8,77.80896,0,2.367647
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,20,0,8,77.44917,0,0.000000
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,21,0,8,77.44917,0,0.000000
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,22,0,8,77.52246,0,0.000000


In [23]:
test_x = test.drop(['date_time'], axis=1)

In [24]:
test_x = test_x.rename({'기온(°C)': 'feature_2', '풍속(m/s)':'feature_3', '습도(%)':'feature_4', '강수량(mm, 6시간)':'feature_5', '일조(hr, 3시간)':'feature_6', '비전기냉방설비운영':'feature_7', '태양광보유':'feature_8', 'time':'feature_9', 'weekday':'feature_10', 'month':'feature_11', 'DI':'feature_12', 'feel_DI':'feature_13', '태양광':'feature_14'}, axis='columns')

In [25]:
test_x

,num,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14
0,1,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,0,1,8,80.982240,1,0.000000
1,1,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,1,1,8,80.653017,0,0.000000
2,1,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,2,1,8,80.323793,0,0.000000
3,1,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,1,8,79.994570,0,0.000000
4,1,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,4,1,8,79.271747,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,19,0,8,80.088017,0,4.061538
10076,60,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,20,0,8,80.154933,0,4.061538
10077,60,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,21,0,8,80.221850,1,4.061538
10078,60,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,22,0,8,80.221850,0,4.061538


# Training

In [26]:
pred = pd.DataFrame()
col_x = ['feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
         'feature_9', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14']
col_y = 'feature_1'


for num in tqdm(range(1, 61)):
    sub_train_x = train_x[train_x['num'] == num]
    sub_test_x = test_x[test_x['num'] == num]
    
    h2o_train_x = h2o.H2OFrame(sub_train_x)
    h2o_test_x = h2o.H2OFrame(sub_test_x)

    aml = H2OAutoML(max_models = 25, seed = 42)
    
    aml.train(x = col_x, y = col_y, training_frame = h2o_train_x)
    
    lb = aml.leaderboard
    print(lb.head(rows=lb.nrows))
    
    aml.leader.download_mojo(path="./mojo/")

    h2o.save_model(aml.leader, path="./save_models/aml_model_{}".format(num))
    
    tmp_pred = aml.predict(h2o_test_x)
    tmp_pred_df = h2o.as_list(tmp_pred)
    pred = pd.concat([pred, tmp_pred_df], axis=0)
    print("num : ", num)
    print("pred : ", len(pred))

  0%|                                                                                           | 0/60 [00:00<?, ?it/s]


22:37:16.678: AutoML: XGBoost is not available; skipping it.



model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_4_AutoML_20210530_223716,1951.55,44.1764,1951.55,28.2218,0.00524345
GBM_grid__1_AutoML_20210530_223716_model_3,2040.08,45.1673,2040.08,28.9922,0.00536155
DRF_1_AutoML_20210530_223716,2068.84,45.4846,2068.84,28.8168,0.00540163
GBM_3_AutoML_20210530_223716,2072.88,45.5289,2072.88,29.1328,0.00540531
GBM_1_AutoML_20210530_223716,2076.78,45.5717,2076.78,28.9259,0.00540621
GBM_2_AutoML_20210530_223716,2109.94,45.9341,2109.94,29.4859,0.0054509
GBM_grid__1_AutoML_20210530_223716_model_1,2135.86,46.2154,2135.86,29.3913,0.00549115
GBM_grid__1_AutoML_20210530_223716_model_5,2179.72,46.6875,2179.72,30.2528,0.00554373
GBM_grid__1_AutoML_20210530_223716_model_2,2187.9,46.7749,2187.9,29.8413,0.00556076
XRT_1_AutoML_20210530_223716,2222.31,47.1414,2222.31,30.9017,0.00559919



num :  1
pred :  168


  2%|█▎                                                                            | 1/60 [16:51<16:35:00, 1011.87s/it]

...

 98%|███████████████████████████████████████████████████████████████████████████▋ | 59/60 [18:40:11<17:13, 1033.26s/it]


17:17:27.755: AutoML: XGBoost is not available; skipping it.



model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20210531_171727,18739.5,136.892,18739.5,99.641,0.0412418
StackedEnsemble_AllModels_AutoML_20210531_171727,18754.8,136.948,18754.8,99.5984,0.041254
GBM_grid__1_AutoML_20210531_171727_model_1,18996.7,137.829,18996.7,99.8687,0.04151
GBM_4_AutoML_20210531_171727,19107.9,138.231,19107.9,99.591,0.0416077
GBM_grid__1_AutoML_20210531_171727_model_2,19136.3,138.334,19136.3,101.02,0.0416601
GBM_3_AutoML_20210531_171727,19154,138.398,19154,101.09,0.0416802
GBM_2_AutoML_20210531_171727,19814.3,140.763,19814.3,102.896,0.0423656
GBM_grid__1_AutoML_20210531_171727_model_4,20170.6,142.023,20170.6,105.157,0.042902
GBM_grid__1_AutoML_20210531_171727_model_3,20194.3,142.107,20194.3,102.979,0.042724
GBM_grid__1_AutoML_20210531_171727_model_7,20196.3,142.114,20196.3,104.45,0.0427802



num :  60
pred :  10080


100%|█████████████████████████████████████████████████████████████████████████████| 60/60 [18:59:28<00:00, 1139.47s/it]


In [27]:
"""saved_model = h2o.load_model('./save_models/aml_model_1/GBM_grid__1_AutoML_20210530_021937_model_2')

sub_test_x = test_x[test_x['num'] == 1]
h2o_test_x = h2o.H2OFrame(sub_test_x)
tmp_pred = saved_model.predict(h2o_test_x)
tmp_pred_df = h2o.as_list(tmp_pred)

tmp_pred_df"""

"saved_model = h2o.load_model('./save_models/aml_model_1/GBM_grid__1_AutoML_20210530_021937_model_2')\n\nsub_test_x = test_x[test_x['num'] == 1]\nh2o_test_x = h2o.H2OFrame(sub_test_x)\ntmp_pred = saved_model.predict(h2o_test_x)\ntmp_pred_df = h2o.as_list(tmp_pred)\n\ntmp_pred_df"

# Submission

In [28]:
pred

,predict
0,8725.081872
1,8719.827350
2,8715.432920
3,8708.157275
4,8674.958497
...,...
163,4233.530923
164,4185.810948
165,3936.108330
166,3782.079877


In [29]:
submission = pd.read_csv('./submission/sample_submission.csv')

In [30]:
submission['answer'] = pred['predict'].values

In [31]:
submission

,num_date_time,answer
0,1 2020-08-25 00,8725.081872
1,1 2020-08-25 01,8719.827350
2,1 2020-08-25 02,8715.432920
3,1 2020-08-25 03,8708.157275
4,1 2020-08-25 04,8674.958497
...,...,...
10075,60 2020-08-31 19,4233.530923
10076,60 2020-08-31 20,4185.810948
10077,60 2020-08-31 21,3936.108330
10078,60 2020-08-31 22,3782.079877


In [32]:
submission['answer'].describe()

count    10080.000000
mean      2546.598236
std       2184.555073
min        274.359171
25%       1207.583095
50%       1899.404760
75%       3033.245329
max      17434.703534
Name: answer, dtype: float64

In [33]:
submission.to_csv('./submission/AutoML_{}.csv'.format(version), index=False)